## Part 1. Importing and Paths

In [4]:
# import all required Python packages:
from __future__ import print_function
import skimage.io as io
import numpy as np
import os, shutil


from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.externals import joblib

In [20]:
# set up your directories 
rootdir = ""


# Based on the root directory for a Mac

# Landsat scene
imagePath = ""


# Rasterized form of training grounds
trainingPath = ""


# Where to store the training model
modelPath = ""


# Where to store the final image
classPath = ""

## Part 2. Training

In [16]:
def training(rootdir, imagePath, trainingPath, ModelPath ):
    """
    Training grounds and rasters have to have the same dimensions. 
    Use numpy or ArcGIS to fix this if you run into problems. 
    
    Weights and representation of training grounds have a large impact 
    on the pixel classification objective of this code. 
    
    """
    
    
    # Path to the tif image
    raster = rootdir + imagePath
   
    # path to your corresponding pixel samples (training data) 
    samples = rootdir + trainingPath

    # read in raster
    img_ds = io.imread(raster)
    
    # convert raster to a 16-bit numpy array 
    img = np.array(img_ds, dtype='int16')

    #do the same with your sample pixels 
    roi_ds = io.imread(samples)   
    roi = np.array(roi_ds, dtype='int16')  
  
    # read in your labels
    labels = np.unique(roi[roi > 0]) 
    
    print('The training data include {n} classes: {classes}'.format(n=labels.size, classes=labels))

    # compose your X,Y data (dataset - training data)     
    X = img[roi > 0, :]
    print("X", X)
   
    Y = roi[roi > 0]     
    print("Y",Y)

    # assign class weights (class 1 has the weight 3, etc.)
    weights = {1:1, 2:2, 3:2, 4:3}

    # build your Random Forest Classifier 
    # for more information: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    # n_estimators = the number of trees in the forest 
    # gotta find the sweet spot where there is not much more return based on the computation time and resources needed
    
    
    rf = RandomForestClassifier(class_weight = weights, n_estimators = 8, criterion = 'gini', max_depth = 4, 
                                min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', 
                                bootstrap = True, oob_score = True, n_jobs = -1, random_state = None, verbose = True)  




    #  now fit your training data with the original dataset
    rf = rf.fit(X,Y)

    # export your Random Forest / Gradient Boosting Model     
    model = rootdir + modelPath
    joblib.dump(rf, model)


# Call the function
training(rootdir, imagePath, trainingPath, modelPath)


The training data include 5 classes: [1 2 3 4 5]
X [[255 151 178 ...,  43  82  24]
 [255 146 172 ...,  46  82  25]
 [255 140 168 ...,  47  82  25]
 ..., 
 [255 229 255 ..., 227  65 136]
 [255 232 255 ..., 233  65 138]
 [255 231 255 ..., 234  65 138]]
Y [2 2 2 ..., 1 1 1]


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    2.9s finished
/Users/colemanshepard/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/colemanshepard/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


## Part 3. Classification

In [21]:
def classification(rootdir,modelPath,imagePath, classPath):
    
    # Path to the tif image
    raster = rootdir + imagePath
    
    # Read worldfile of original dataset
    tfw_old = str(raster.split(".tif")[0]) + ".tfw"     

    # Read Data    
    img_ds = io.imread(raster)   
    img = np.array(img_ds, dtype='int16')    

    # call your random forest model
    rf = rootdir + modelPath          
    clf = joblib.load(rf)    
    print("Stage 1")
    
    # Classification of array and save as image (23 refers to the number of multitemporal NDVI bands in the stack) 
    new_shape = (img.shape[0] * img.shape[1], img.shape[2]) 
    img_as_array = img[:, :, :7].reshape(new_shape)   
    print("Stage 2")
    
    # Prediction step
    class_prediction = clf.predict(img_as_array) 
    class_prediction = class_prediction.reshape(img[:, :, 0].shape)  
    print("Stage 3")
    
    # now export your classificaiton
    classification = rootdir  + classPath 
    io.imsave(classification, class_prediction)    
    print("Stage 4")
    
    # Assign Worldfile to classified image    
    tfw_new = classification.split(".tif")[0] + ".tfw"   
    shutil.copy(tfw_old, tfw_new)
    print("Stage 5")
    
# Call the function
classification(rootdir, modelPath, imagePath, classPath)

Stage 1
Stage 2


[Parallel(n_jobs=4)]: Done   8 out of   8 | elapsed:   13.1s finished


Stage 3


/Users/colemanshepard/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/colemanshepard/Desktop/Adv Remote Sensing/Adv Remote Sensing Project with Jacob/Path_Class/newClassificationGeocomputingTest2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


Stage 4
Stage 5
